In [22]:
import h5py
import numpy as np
import pandas as pd

def extract_filtered_data(file_path, lat_range=(39, 41), lon_range=(140, 142), max_entries=4500):
    """
    Extract and filter HDF5 data for specified attributes within coordinate ranges.
    
    Args:
        file_path (str): Path to HDF5 file
        lat_range (tuple): Range of latitude (min, max)
        lon_range (tuple): Range of longitude (min, max)
        max_entries (int): Maximum number of entries to process
        
    Returns:
        pandas.DataFrame: DataFrame containing filtered data with all attributes
    """
    # Define all attributes to extract
    attributes = [
        'year', 'month', 'day', 'hour', 'min', 'sec', 'recno', 'kindat', 'kinst',
        'ut1_unix', 'ut2_unix', 'pierce_alt', 'gps_site', 'sat_id', 'gnss_type',
        'gdlatr', 'gdlonr', 'los_tec', 'dlos_tec', 'tec', 'azm', 'elm',
        'gdlat', 'glon', 'rec_bias', 'drec_bias'
    ]
    
    with h5py.File(file_path, 'r') as hdf:
        # Get the dataset
        dataset = hdf['Data/Table Layout']
        
        # Read only the first max_entries
        data = dataset[:max_entries]
        
        # Extract coordinates for filtering
        lats = data['gdlatr']  # receiver latitude
        lons = data['gdlonr']  # receiver longitude
        
        # Create mask for the coordinate ranges
        mask = (lats >= lat_range[0]) & (lats <= lat_range[1]) & \
               (lons >= lon_range[0]) & (lons <= lon_range[1])
        
        # Create dictionary to store filtered data
        filtered_data = {}
        
        # Extract each attribute and apply mask
        for attr in attributes:
            if attr in data.dtype.names:
                filtered_values = data[attr][mask]
                
                # Convert bytes to strings for string columns
                if filtered_values.dtype.kind == 'S':
                    filtered_values = [val.decode('utf-8') for val in filtered_values]
                
                filtered_data[attr] = filtered_values
        
        # Create pandas DataFrame
        df = pd.DataFrame(filtered_data)
        
        # Rename 'min' and 'sec' columns to 'minute' and 'second' for datetime conversion
        df = df.rename(columns={'min': 'minute', 'sec': 'second'})
        
        try:
            # Add datetime column for convenience
            df['datetime'] = pd.to_datetime(
                df[['year', 'month', 'day', 'hour', 'minute', 'second']]
            )
        except ValueError as e:
            print(f"Warning: Could not create datetime column: {e}")
            # If datetime creation fails, we still want to return the DataFrame
            pass
        
        # Rename back to original column names if you prefer
        df = df.rename(columns={'minute': 'min', 'second': 'sec'})
        
        return df

# Example usage
file_path = r'/mnt/c/Users/fadri_oudurqw/Downloads/los_20110311.001_Jap_Tsu.h5'
df = extract_filtered_data(file_path, max_entries=100_000)

# Export the DataFrame to a CSV file
df.to_csv('data/data_csvs/Japan_Tsunami_11Mar2011.csv', index=False)

# Example usage with full args*
# df = extract_filtered_data(file_path, lat_range=(39, 41), lon_range=(140, 142), max_entries=100)

# Display information about the resulting DataFrame
print("\nDataFrame Info:")
print(df.info())

print("\nFirst few rows:")
print(df.head())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   year        54 non-null     int64         
 1   month       54 non-null     int64         
 2   day         54 non-null     int64         
 3   hour        54 non-null     int64         
 4   min         54 non-null     int64         
 5   sec         54 non-null     int64         
 6   recno       54 non-null     int64         
 7   kindat      54 non-null     int64         
 8   kinst       54 non-null     int64         
 9   ut1_unix    54 non-null     float64       
 10  ut2_unix    54 non-null     float64       
 11  pierce_alt  54 non-null     float64       
 12  gps_site    54 non-null     object        
 13  sat_id      54 non-null     int64         
 14  gnss_type   54 non-null     object        
 15  gdlatr      54 non-null     float64       
 16  gdlonr     

In [ ]:
# import h5py

# def explore_h5(file_path):
#     """
#     Explore the structure of an HDF5 file and print all groups and datasets with their paths.
    
#     Parameters:
#     -----------
#     file_path : str
#         Path to the HDF5 file
#     """
#     def print_structure(name, obj):
#         """Callback function to print the structure"""
#         indent = '    ' * name.count('/')
#         if isinstance(obj, h5py.Dataset):
#             print(f"{indent}Dataset: {name}")
#             print(f"{indent}    Shape: {obj.shape}")
#             print(f"{indent}    Type: {obj.dtype}")
#         elif isinstance(obj, h5py.Group):
#             print(f"{indent}Group: {name}")
    
#     try:
#         with h5py.File(file_path, 'r') as f:
#             print(f"File: {file_path}")
#             print("\nStructure:")
#             # Recursively visit all groups and datasets
#             f.visititems(print_structure)
            
#     except Exception as e:
#         print(f"Error occurred: {str(e)}")

# # Example usage:
# explore_h5('/mnt/c/Users/fadri_oudurqw/Downloads/los_20160326.001.h5')

# def list_datasets(file_path):
#     """
#     List all dataset paths in an HDF5 file.
    
#     Parameters:
#     -----------
#     file_path : str
#         Path to the HDF5 file
        
#     Returns:
#     --------
#     list
#         List of dataset paths
#     """
#     dataset_paths = []
    
#     def collect_datasets(name, obj):
#         if isinstance(obj, h5py.Dataset):
#             dataset_paths.append(name)
    
#     with h5py.File(file_path, 'r') as f:
#         f.visititems(collect_datasets)
    
#     return dataset_paths

# # Example usage:
# # paths = list_datasets('your_file.h5')
# # print("\nDataset paths:", *paths, sep='\n')

File: /mnt/c/Users/fadri_oudurqw/Downloads/los_20160326.001.h5

Structure:
Group: Data
    Dataset: Data/Table Layout
        Shape: (111787628,)
        Type: [('year', '<i8'), ('month', '<i8'), ('day', '<i8'), ('hour', '<i8'), ('min', '<i8'), ('sec', '<i8'), ('recno', '<i8'), ('kindat', '<i8'), ('kinst', '<i8'), ('ut1_unix', '<f8'), ('ut2_unix', '<f8'), ('pierce_alt', '<f8'), ('gps_site', 'S4'), ('sat_id', '<i8'), ('gdlatr', '<f8'), ('gdlonr', '<f8'), ('los_tec', '<f8'), ('dlos_tec', '<f8'), ('tec', '<f8'), ('azm', '<f8'), ('elm', '<f8'), ('gdlat', '<f8'), ('glon', '<f8'), ('rec_bias', '<f8'), ('drec_bias', '<f8')]
Group: Metadata
    Dataset: Metadata/Data Parameters
        Shape: (25,)
        Type: [('mnemonic', 'S10'), ('description', 'S50'), ('isError', '<i8'), ('units', 'S7'), ('category', 'S36')]
    Dataset: Metadata/Experiment Parameters
        Shape: (14,)
        Type: [('name', 'S20'), ('value', 'S46')]
    Dataset: Metadata/Independent Spatial Parameters
        Shape:

In [20]:
import h5py
import pandas as pd

def extract_h5_measurements(file_path, dataset_path, n_measurements=500):
    """
    Extract the first n measurements from an HDF5 file into a pandas DataFrame.
    
    Parameters:
    -----------
    file_path : str
        Path to the HDF5 file
    dataset_path : str
        Path to the dataset within the HDF5 file
    n_measurements : int, optional
        Number of measurements to extract (default: 500)
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing the extracted measurements
    """
    try:
        with h5py.File(file_path, 'r') as f:
            # Get the dataset
            dataset = f[dataset_path]
            
            # Extract first n_measurements
            data = dataset[:n_measurements]
            # Filter for stations with certain gdlon and gdlat
            filtered_data = data[(data['gdlonr'] >= 140) & (data['gdlonr'] <= 142) & 
                                 (data['gdlatr'] >= 39) & (data['gdlatr'] <= 41)]
            # Convert to DataFrame
            df = pd.DataFrame(filtered_data)
            
            return df
            
    except KeyError:
        print(f"Dataset {dataset_path} not found in file")
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        
# Example usage:
df = extract_h5_measurements('/mnt/c/Users/fadri_oudurqw/Downloads/los_20160326.001.h5', 'Data/Table Layout', n_measurements=100_000_00)
df

,year,month,day,hour,min,sec,recno,kindat,kinst,ut1_unix,...,gdlonr,los_tec,dlos_tec,tec,azm,elm,gdlat,glon,rec_bias,drec_bias
0,2016,3,26,0,0,0,0,3505,8000,1.458950e+09,...,141.132828,23.154556,0.374467,12.409058,134.274796,27.289396,35.299515,145.801300,-13.615938,1.300858
1,2016,3,26,0,0,0,0,3505,8000,1.458950e+09,...,141.132828,16.347767,0.373366,11.964964,57.927349,44.170170,40.664635,144.457001,-13.615938,1.300858
2,2016,3,26,0,0,0,0,3505,8000,1.458950e+09,...,141.132828,4.626015,0.202011,4.393531,-11.427274,70.761436,40.141891,140.865005,-13.615938,1.300858
3,2016,3,26,0,0,0,0,3505,8000,1.458950e+09,...,141.132828,18.365589,2.093730,11.724884,-50.269512,35.884354,41.573429,137.101181,-13.615938,1.300858
4,2016,3,26,0,0,0,0,3505,8000,1.458950e+09,...,141.132828,13.004891,0.908312,12.683059,160.805191,76.542198,38.450500,141.431488,-13.615938,1.300858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753,2016,3,26,2,4,30,249,3505,8000,1.458958e+09,...,141.132828,16.955704,0.202011,12.299959,102.312950,43.569721,38.411732,144.929672,-13.615938,1.300858
1754,2016,3,26,2,4,30,249,3505,8000,1.458958e+09,...,141.132828,21.247169,2.093730,18.908945,-127.848763,61.310795,38.115734,139.506317,-13.615938,1.300858
1755,2016,3,26,2,4,30,249,3505,8000,1.458958e+09,...,141.132828,53.092110,0.908312,22.182133,165.859543,16.990219,31.303854,143.416473,-13.615938,1.300858
1756,2016,3,26,2,4,30,249,3505,8000,1.458958e+09,...,141.132828,17.443150,0.488239,15.599589,37.868587,61.900990,40.368431,142.403595,-13.615938,1.300858
